In [1]:
daily_returns = sc.textFile("s3n://msan-usf-gproject/daily_returns.csv")
header = daily_returns.first()

rdd = daily_returns.filter(lambda x: x != header)

In [2]:
def toDouble(x):
    try:
        return float(x)
    except ValueError:
        return 0.0
    
rddSplit = rdd.map(lambda x: x.split(','))
rddDoubles = rddSplit.map(lambda x: tuple([toDouble(val) for val in x]))
rddDoubles.take(1)

[(10.0,
  0.0,
  0.370326220989,
  -0.00631639920175,
  0.222831085324,
  -0.213030129671,
  0.729276776314,
  -0.335633248091,
  0.113292053342,
  1.62123823166,
  -0.179403930902,
  0.0,
  -0.0721078515053,
  0.249186635017,
  0.0244014300406,
  -0.127942487597,
  0.0,
  1.41274225712,
  -0.0295753479004,
  1.26524603367,
  -0.0557470582426,
  1.59225594997,
  -0.285275280476,
  -0.212887898088,
  0.40418022871,
  0.116902828217,
  0.197589740157,
  -0.197454556823,
  -0.195023342967,
  -0.0598861537874,
  -0.0211989730597,
  -0.0135547351092,
  -0.236370578408,
  -0.253357321024,
  0.575561583042,
  0.283947110176,
  -0.00620764028281,
  0.616350948811,
  -0.0345781445503,
  0.732652306557,
  -0.00243225856684,
  -0.0925076529384,
  -0.0721848458052,
  0.447295695543,
  -0.194317787886,
  0.264373421669,
  0.01876296103,
  0.0,
  -0.145278140903,
  -0.108546897769,
  0.148189216852,
  -0.226942017674,
  0.262863576412,
  -0.252933114767,
  -0.113264843822,
  0.180559366941,
  -0.033

In [3]:
df = sqlContext.createDataFrame(rddDoubles,  header.split(','))
df.select('id', 'timestamp').show()

+----+---------+
|  id|timestamp|
+----+---------+
|10.0|      0.0|
|11.0|      0.0|
|12.0|      0.0|
|25.0|      0.0|
|26.0|      0.0|
|27.0|      0.0|
|31.0|      0.0|
|38.0|      0.0|
|39.0|      0.0|
|40.0|      0.0|
|41.0|      0.0|
|43.0|      0.0|
|44.0|      0.0|
|49.0|      0.0|
|54.0|      0.0|
|59.0|      0.0|
|60.0|      0.0|
|62.0|      0.0|
|63.0|      0.0|
|68.0|      0.0|
+----+---------+
only showing top 20 rows



In [4]:
df_train = df.where('timestamp < 906')
df_test = df.where('timestamp >= 906')

In [5]:
from pyspark.ml.feature import VectorAssembler
va_train = VectorAssembler(outputCol="features", inputCols=df_train.columns[2:-1])
va_test = VectorAssembler(outputCol="features", inputCols=df_test.columns[2:-1])
train = va_train.transform(df_train).select("features", "y").cache()
test = va_test.transform(df_test).select("features", "y").cache()

In [6]:
train = train.withColumnRenamed(existing='y', new='label')
test = test.withColumnRenamed(existing='y', new='label')

In [7]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(numTrees=100, maxDepth=5)
rfmodel = rf.fit(train)

In [8]:
rfpredcitions = rfmodel.transform(test)

In [12]:
pred = rfpredcitions.select("prediction").rdd.map(lambda r: r[0]).collect()

In [13]:
labels = rfpredcitions.select("label").rdd.map(lambda r: r[0]).collect()

In [14]:
import numpy as np

pred = np.array(pred)
labels = np.array(labels)

def rvalue(pred, y):
    mu = np.mean(y)
    rsquare = 1 - sum( (y - pred)**2 ) / sum ( (y - mu )**2)
    r = np.sign(rsquare) * np.sqrt( abs(rsquare) )
    
    return r

print rvalue(pred, labels)

0.0254168611149
